In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import os
import numpy as np
import pandas as pd 
import sys

workingdirectory = os.popen('git rev-parse --show-toplevel').read()[:-1]
sys.path.append(workingdirectory)
os.chdir(workingdirectory)

import allensdk.core.json_utilities as ju
from allensdk.core.mouse_connectivity_cache import MouseConnectivityCache

from mcmodels.core import VoxelModelCache
from mcmodels.core.utils import get_structure_id,get_ordered_summary_structures
from mcmodels.core.connectivity_data import get_connectivity_data
from mcmodels.core.utils import get_leaves_ontologicalorder
from mcmodels.utils import get_aligned_ids, fix_pdcsv

/Users/samsonkoelle/miniconda3/envs/mcm_class/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.linear_model.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.linear_model. Anything that cannot be imported from sklearn.linear_model is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/samsonkoelle/miniconda3/envs/mcm_class/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [3]:
#read data
TOP_DIR = workingdirectory
INPUT_JSON = workingdirectory + '/data/meta/input_011520.json'
EXPERIMENTS_EXCLUDE_JSON = workingdirectory + '/data/meta/experiments_exclude.json'

input_data = ju.read(INPUT_JSON)
manifest_file = input_data.get('manifest_file')
manifest_file = os.path.join(TOP_DIR, manifest_file)
experiments_exclude = ju.read(EXPERIMENTS_EXCLUDE_JSON)

#NOTE (Sam): hyperparameters are loaded from the output directory
cache = VoxelModelCache(manifest_file=manifest_file)
major_structures = input_data.get('structures')
major_structure_ids = [get_structure_id(cache, s) for s in major_structures]
data_info = pd.read_excel(workingdirectory + '/data/meta/Whole Brain Cre Image Series_curation only.xlsx', 'all datasets curated_070919pull')
data_info.set_index("id", inplace=True)

#NOTE (Sam): switched from v2 structures to v3
#ontological_order = get_ordered_summary_structures(cache)
ontological_order = get_ordered_summary_structures(cache,167587189)

#NOTE (Sam): due to redundancy in ccfv3, its necessary to remove 'MDRNv', 'MDRNd' from summary structures (include as leafs of MDRN)
#we retain ordering
ontological_order = ontological_order[~np.in1d(ontological_order,[1098, 1107])]

#NOTE (Sam): this was (seemingly mistakenly) mouse_connectivity_manifest.json in an earlier version
mcc = MouseConnectivityCache(manifest_file = workingdirectory + '/data/meta/new_manifest.json')
st = mcc.get_structure_tree()
ai_map = st.get_id_acronym_map()
ia_map = {value: key for key, value in ai_map.items()}

cre = None
eid_set=None
high_res=False
threshold_injection = False

COARSE_STRUCTURE_SET_ID = 2
DEFAULT_STRUCTURE_SET_IDS = tuple([COARSE_STRUCTURE_SET_ID])
tree = cache.get_structure_tree()
default_structures = tree.get_structures_by_set_id(DEFAULT_STRUCTURE_SET_IDS)
default_structure_ids = [st['id'] for st in default_structures if st['id'] != 934]
folder =  '/Users/samsonkoelle/mouse_connectivity_models/data/raw/'

annotation/ccf_2017


In [4]:
#NOTE (Sam): this will download the data if necessary 
connectivity_data = get_connectivity_data(cache, major_structure_ids, experiments_exclude, remove_injection = False, structure_set_id=167587189, folder = folder)
connectivity_data.get_injection_hemisphere_ids()
connectivity_data.align()
connectivity_data.get_centroids()
connectivity_data.get_data_matrices(default_structure_ids)
connectivity_data.get_crelines(data_info)
connectivity_data.ai_map = ai_map
connectivity_data.get_summarystructures(data_info)


512
703
1089
1097
315
313
354
698
771
803
477
549


In [5]:
ontological_order_leaves = get_leaves_ontologicalorder(connectivity_data, ontological_order)


In [8]:
np.isin(269, ontological_order_leaves)

array(True)

In [10]:
leafs[315]

NameError: name 'leafs' is not defined

In [6]:
np.save(workingdirectory + '/data/meta/default_structures', default_structures)
np.save(workingdirectory + '/data/meta/ontological_order_v3', ontological_order)
np.save(workingdirectory + '/data/meta/major_structures',major_structures)
np.save(workingdirectory + '/data/meta/major_structure_ids',major_structure_ids)
np.save(workingdirectory + '/data/meta/default_structure_ids', default_structure_ids)
np.save(workingdirectory + '/data/meta/ontological_order_leaves_v3', ontological_order_leaves)

In [6]:
# wt_conn = pd.read_csv(workingdirectory + '/paper/connectivities/elsumsurfleafsmth_v3_leafsum_wt.csv', header = [0,1])
# wt_conn = fix_pdcsv(wt_conn)
# targ_sum = np.asarray(list(wt_conn.columns))[:,1]
# targ_sum_id = np.asarray([ai_map[targ_sum[i]] for i in range(len(targ_sum))])
# targ_sum_major = get_aligned_ids(targ_sum_id,major_structure_ids)
# source_leaf_major = get_aligned_ids(ontological_order_leaves,major_structure_ids)
# source_leaf_major_names = np.asarray([ia_map[source_leaf_major[i]] for i in range(len(source_leaf_major))])
# targ_sum_major_names = np.asarray([ia_map[targ_sum_major[i]] for i in range(len(targ_sum_major))])
# np.save(workingdirectory + '/paper/info/source_leaf_major_names', source_leaf_major_names)
# np.save(workingdirectory + '/paper/info/targ_sum_major_names', targ_sum_major_names)